### Pyspark particionamento

Particionamento é o processo de dividir os dados em blocos menores (partições) que podem ser processados em paralelo por diferentes workers do cluster. No Spark, cada partição é processada por uma tarefa (task).

In [1]:
import multiprocessing
multiprocessing.cpu_count()

8

> Saber quantos núcleos de CPU você tem é essencial pra definir um bom número de partições no Spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [3]:
spark = SparkSession\
            .builder\
            .appName("particao")\
            .master("local[8]")\
            .getOrCreate()

In [4]:
spark

In [5]:
spark.sparkContext.defaultParallelism

8

> **defaultParallelism** : Retorna o número padrão de tarefas em paralelo, geralmente igual ao número de núcleos quando local[*] é usado

Lendo o arquivo no spark

```python


dict_ = dict(
    header='True',
    inferSchema='True'
)
# Aqui pode definir o typo de cada dados
schema = StructType([
    StructField(name = 'id', dataType = IntegerType(), nullable = True),
    StructField(name = 'sexo', dataType = StringType(), nullable = True),
    StructField(name = 'idade', dataType = IntegerType(), nullable = True),
    StructField(name = 'diabetes', dataType = IntegerType(), nullable = True),
])

data = spark.read\
        .options(**dict_)\
        .format('csv')\
        .load(path)
```

In [6]:
path = 'C:\\Users\\mateu\\Documents\\MEGA\\Projetos\\HELPS\\MiniBook\\DataAnalise\\dataset\\dados.csv'

In [7]:
dict_ = dict(
    header='True',
    inferSchema='True'
)
data = spark.read.options(**dict_)\
        .format('csv')\
        .load(path)

In [8]:
data.show(4)

+----------+----+--------------------+-----+-----+-------+--------+----------+
|        id| ano|                nome|preço|prime|empresa|desconto|    estado|
+----------+----+--------------------+-----+-----+-------+--------+----------+
|9001763654|1973|       Evelyn Fogaça| 3587|false| nissan|    0.05|   Alagoas|
|8096557597|2002|Sr. Davi Lucca Re...| 1289|false| nissin|     0.0|Pernambuco|
|6451624339|2007|     Matheus Freitas| 1343|false| toyota|    0.05|     Piauí|
|2421207048|1998|Sr. Marcos Vinici...| 4538|false| nissan|     0.1| São Paulo|
+----------+----+--------------------+-----+-----+-------+--------+----------+
only showing top 4 rows



Saber o numero de partições no meu dataframe

In [9]:
data.rdd.getNumPartitions()

8

Saber que repartição pertence a aquela instancia

In [10]:
data.withColumn('prime', F.spark_partition_id()).select(F.col('prime')).distinct().show()

+-----+
|prime|
+-----+
|    0|
|    1|
|    2|
|    3|
|    4|
|    5|
|    6|
|    7|
+-----+



### Operações no Pyspark

`Select`

In [11]:
data.select(F.col('empresa')).show(2)

+-------+
|empresa|
+-------+
| nissan|
| nissin|
+-------+
only showing top 2 rows



`where`

In [12]:
data.where(
    F.col('empresa').like('%i%')
).show(3)

+----------+----+--------------------+-----+-----+-------+--------+----------+
|        id| ano|                nome|preço|prime|empresa|desconto|    estado|
+----------+----+--------------------+-----+-----+-------+--------+----------+
|9001763654|1973|       Evelyn Fogaça| 3587|false| nissan|    0.05|   Alagoas|
|8096557597|2002|Sr. Davi Lucca Re...| 1289|false| nissin|     0.0|Pernambuco|
|2421207048|1998|Sr. Marcos Vinici...| 4538|false| nissan|     0.1| São Paulo|
+----------+----+--------------------+-----+-----+-------+--------+----------+
only showing top 3 rows



In [13]:
data.where(
    F.col('empresa').like('%i%') &
    F.col('prime') == 'true'
).show(3)


+----------+----+--------------------+-----+-----+-------+--------+----------------+
|        id| ano|                nome|preço|prime|empresa|desconto|          estado|
+----------+----+--------------------+-----+-----+-------+--------+----------------+
|6107140111|2017|   João Lucas da Luz| 1519| true| nissin|    0.05|  Espírito Santo|
|7646075356|1986|     Beatriz da Cruz| 1745| true| nissan|    0.05|  Santa Catarina|
|6987763002|2002|Dra. Pietra Gonça...| 3031| true| nissin|     0.0|Distrito Federal|
+----------+----+--------------------+-----+-----+-------+--------+----------------+
only showing top 3 rows



In [14]:
data.where(
    F.col('empresa').like('%i%') |
    F.col('empresa').like('%toyota%')
).show(3)

+----------+----+--------------------+-----+-----+-------+--------+----------+
|        id| ano|                nome|preço|prime|empresa|desconto|    estado|
+----------+----+--------------------+-----+-----+-------+--------+----------+
|9001763654|1973|       Evelyn Fogaça| 3587|false| nissan|    0.05|   Alagoas|
|8096557597|2002|Sr. Davi Lucca Re...| 1289|false| nissin|     0.0|Pernambuco|
|6451624339|2007|     Matheus Freitas| 1343|false| toyota|    0.05|     Piauí|
+----------+----+--------------------+-----+-----+-------+--------+----------+
only showing top 3 rows



`agrupamento`

In [15]:
data.groupBy(F.col('empresa')).count().show()

+--------+-------+
| empresa|  count|
+--------+-------+
|  nissin|2498159|
|  toyota|2498563|
|brastemp|2502791|
|  nissan|2500487|
+--------+-------+



In [16]:
data.groupBy(F.col('prime'))\
    .agg(
        F.sum(F.col('preço')).alias('total'),\
        F.avg(F.col('preço')).alias('media'),\
        F.max(F.col('preço')).alias('maior'),\
        F.min(F.col('preço')).alias('menor'),\
        F.stddev(F.col('preço')).alias('desvio')
    ).show()

+-----+-----------+------------------+-----+-----+------------------+
|prime|      total|             media|maior|menor|            desvio|
+-----+-----------+------------------+-----+-----+------------------+
| true| 7497538684|2999.1978248277496| 4999| 1000|1155.0704546123302|
|false|22497528263|2999.6096429779022| 4999| 1000|1154.9842401628714|
+-----+-----------+------------------+-----+-----+------------------+



In [17]:
data.groupBy(F.col('prime'), F.col('empresa')).count().show()

+-----+--------+-------+
|prime| empresa|  count|
+-----+--------+-------+
| true|  nissin| 624952|
| true|  toyota| 623958|
| true|  nissan| 624815|
|false|  nissan|1875672|
|false|  nissin|1873207|
|false|  toyota|1874605|
| true|brastemp| 626123|
|false|brastemp|1876668|
+-----+--------+-------+



In [18]:
data.show(3)

+----------+----+--------------------+-----+-----+-------+--------+----------+
|        id| ano|                nome|preço|prime|empresa|desconto|    estado|
+----------+----+--------------------+-----+-----+-------+--------+----------+
|9001763654|1973|       Evelyn Fogaça| 3587|false| nissan|    0.05|   Alagoas|
|8096557597|2002|Sr. Davi Lucca Re...| 1289|false| nissin|     0.0|Pernambuco|
|6451624339|2007|     Matheus Freitas| 1343|false| toyota|    0.05|     Piauí|
+----------+----+--------------------+-----+-----+-------+--------+----------+
only showing top 3 rows



`withColumns`

In [19]:
data.withColumns(
    dict(
        empresa=F.col('empresa').substr(1, 3),
        cidade =F.col('estado').substr(1, 3),
        desconto = F.col('preço') * F.col('desconto')
    )
)\
    .select(['empresa', 'estado', 'desconto'])\
        .show(3)

+-------+----------+------------------+
|empresa|    estado|          desconto|
+-------+----------+------------------+
|    nis|   Alagoas|179.35000000000002|
|    nis|Pernambuco|               0.0|
|    toy|     Piauí|             67.15|
+-------+----------+------------------+
only showing top 3 rows



`case`

In [20]:
data.select(
    F.when(F.col('prime') == 'true', 'primeiro').otherwise('outro')
).distinct().show(3)

+-----------------------------------------------------+
|CASE WHEN (prime = true) THEN primeiro ELSE outro END|
+-----------------------------------------------------+
|                                                outro|
|                                             primeiro|
+-----------------------------------------------------+



In [24]:
data.groupBy(F.col('empresa')).mean('preço').show()

+--------+------------------+
| empresa|        avg(preço)|
+--------+------------------+
|  nissin|  2999.59639398453|
|  toyota| 3000.252134927156|
|brastemp|2998.4004825013353|
|  nissan|2999.7794437643547|
+--------+------------------+

